# Clustering Crypto

In [203]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [204]:
# Load the crypto_data.csv dataset.
# crypto_data.csv was retrieved from CryptoCompare: https://min-api.cryptocompare.com/data/all/coinlist
file_path = "Resources/crypto_data.csv"
df = pd.read_csv(file_path, index_col=0)
df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [205]:
# Keep all the cryptocurrencies that are being traded.
df1 =df.loc[df["IsTrading"] == True]
df1

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [206]:
# Drop IsTrading Column
df1 = df1.drop(["IsTrading"], axis = 1)

In [207]:
#  Drop rows with null values
df2 = df1.dropna()

In [208]:
# Keep the rows where coins are mined.
crypto_df = df2.loc[df2["TotalCoinsMined"] > 0 ]
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [209]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = pd.DataFrame({"CoinName" : crypto_df["CoinName"]}, index = crypto_df.index)
names_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [210]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns = ["CoinName"], axis = 1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [211]:
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].astype('float')
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [212]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(data =crypto_df, columns=["Algorithm", "ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2.000000e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,2.500000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1.400223e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
# Standardize the data with StandardScaler().
ssc = StandardScaler()
X_scaled = ssc.fit_transform(X)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [214]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model
pca = PCA(n_components=3)
# Get three principal components
X_pca = pca.fit_transform(X_scaled)

In [215]:
# Create a DataFrame with the three principal components.
pcf_df = pd.DataFrame(
    data=X_pca,
    columns=["PC1", "PC2", "PC3"],
    index = X.index
)
pcf_df.head()

,PC1,PC2,PC3
42,-0.336513,1.054551,-0.574633
404,-0.319852,1.054674,-0.575100
1337,2.304277,1.722395,-0.616285
BTC,-0.143490,-1.333217,0.184782
ETH,-0.153406,-2.035205,0.389474


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [216]:
# Looking for the best K
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcf_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\Rob\anaconda3\envs\MLenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [217]:
# Looking at the elbow curve, it seems that k=4 may be the best value

In [219]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df =pd.concat([crypto_df, pcf_df], axis =1)
clustered_df

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = names_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.336513,1.054551,-0.574633,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.319852,1.054674,-0.575100,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.304277,1.722395,-0.616285,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.143490,-1.333217,0.184782,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.153406,-2.035205,0.389474,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.163549,-1.108105,-0.017963,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.406836,1.172931,-0.551714,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.145760,-2.233182,0.322183,Monero,1
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.151848,-2.035303,0.389454,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.143418,-2.020891,0.377082,ZCash,1


In [225]:
clustered_df["Class"] = clustered_df["Class"].astype(str)
clustered_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
PC1                float64
PC2                float64
PC3                float64
CoinName            object
Class               object
dtype: object

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [226]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color = "Class",
    symbol="Class",
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [230]:
# Create a table with tradable cryptocurrencies.
tradable_cryptos = clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"],
                            sortable=True, 
                            selectable= True)

tradable_cryptos

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [232]:
# Print the total number of tradable cryptocurrencies.
print(f'There is a total of {tradable_cryptos.shape[0]} tradable cryptocurrencies.')

There is a total of 532 tradable cryptocurrencies.


In [236]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mms = MinMaxScaler()
scaled_tradable = mms.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
scaled_tradable

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [241]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
mms_df = pd.DataFrame(scaled_tradable, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
mms_df["CoinName"] = names_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
mms_df["Class"] = clustered_df["Class"]

mms_df = mms_df[['CoinName', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']]
mms_df.head(10)

,CoinName,TotalCoinSupply,TotalCoinsMined,Class
42,42 Coin,4.200000e-11,0.000000,0
404,404Coin,5.320000e-04,0.001066,0
1337,EliteCoin,3.141593e-01,0.029576,0
BTC,Bitcoin,2.100000e-05,0.000018,1
ETH,Ethereum,0.000000e+00,0.000109,1
LTC,Litecoin,8.400000e-05,0.000064,1
DASH,Dash,2.200000e-05,0.000009,0
XMR,Monero,0.000000e+00,0.000017,1
ETC,Ethereum Classic,2.100000e-04,0.000115,1
ZEC,ZCash,2.100000e-05,0.000007,1


In [249]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
mms_df.hvplot.scatter(
    x = "TotalCoinsMined", 
    y = "TotalCoinSupply",
    title = "Cryptocurrencies sorted by mined coins and total supply",
    hover_cols = "CoinName",
    by = "Class"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)